In [31]:
#!/usr/bin/env python
# coding: utf-8


import pandas as pd
import numpy as np
import json
from pandas.api.types import CategoricalDtype

master_df = pd.read_csv('/home/kin672/gentb-summer22/Summarize Sanjana Dataset/full_control_db_7-25.csv')



In [32]:
master_df = master_df.loc[[str(i)[0:1] in ['1','2','3','4','5','6','B'] for i in master_df.Lineage], :]

In [33]:
summarized = master_df.groupby(['Drug','Lineage','Resistant']).count()
summarized = summarized.rename(columns={'ID':'NumStrains'})
summarized.reset_index(inplace=True)

In [51]:
reformat_summarized = pd.DataFrame({'Drug':[], 'Lineage':[], 'Percentage of Strains that are Resistant':[], 'Number of Resistant Strains':[], 'Number of Susceptible Strains':[], 'No Results':[]})
for drug in np.unique(summarized.Drug):
    for lineage in np.unique(summarized.Lineage):
        # Calculate Number Resistant
        num_resistant = summarized.loc[(summarized.Drug == drug) & (summarized.Lineage == lineage) & (summarized.Resistant == 'R'), 'NumStrains']
        if (len(num_resistant) == 1):
            num_resistant = num_resistant.values[0]
        elif (len(num_resistant) == 0):
            num_resistant = 0
        else:
            num_resistant = 'Messed up'
            
        # Calculate Number Susceptible
        num_susceptible = summarized[(summarized.Drug == drug) & (summarized.Lineage == lineage) & (summarized.Resistant == 'S')].NumStrains
        if (len(num_susceptible) == 1):
            num_susceptible = num_susceptible.values[0]
        elif (len(num_susceptible) == 0):
            num_susceptible = 0
        else:
            num_susceptible = 'Messed up'
            
        # Calculate Remaining
        remaining = len(master_df[(master_df.Drug == drug) & (master_df.Lineage == lineage)].ID.unique()) - num_susceptible - num_resistant
            
        # Calculate Percentage Resistant
        if (num_resistant + num_susceptible + remaining) == 0:
            percent = 0
        else:
            percent = num_resistant * 100 / (num_resistant + num_susceptible + remaining)
            
        temp = pd.DataFrame({'Drug':[drug], 'Lineage':[lineage], 'Percentage of Strains that are Resistant':[percent], 'Number of Resistant Strains':[str(num_resistant)], 'Number of Susceptible Strains':[str(num_susceptible)], 'No Results':[str(remaining)]})
        reformat_summarized = pd.concat([reformat_summarized, temp], ignore_index=True)


In [60]:
def main_lineage(isolate):
    lineage = master_df.loc[master_df['ID'] == isolate, 'Lineage'].values[0]
    if ',' in lineage:
        return('Multiple')
    return(lineage[0:1])

def main_lineage_from_lineage(lineage):
    if ',' in lineage:
        return('Multiple')
    return(lineage[0:1])

In [62]:
reformat_summarized['Main Lineage'] = [main_lineage_from_lineage(i) for i in reformat_summarized.Lineage]

In [66]:
reformat_summarized

,Drug,Lineage,Percentage of Strains that are Resistant,Number of Resistant Strains,Number of Susceptible Strains,No Results,Main Lineage
0,amk,1,0.000000,0,1,2,1
1,amk,"1,2.2.1",0.000000,0,0,1,Multiple
2,amk,1.1,0.000000,0,0,1,1
3,amk,1.1.1,0.000000,0,2,34,1
4,amk,1.1.1.1,0.000000,0,2,18,1
...,...,...,...,...,...,...,...
1477,str,5,0.000000,0,2,13,5
1478,str,"5,6,BOV_AFRI",0.000000,0,1,0,Multiple
1479,str,6,0.000000,0,1,1,6
1480,str,"6,BOV_AFRI",2.777778,1,21,14,Multiple


In [72]:
reformat_summarized['Number of Resistant Strains'] = [int(i) for i in reformat_summarized['Number of Resistant Strains']]
reformat_summarized['Number of Susceptible Strains'] = [int(i) for i in reformat_summarized['Number of Susceptible Strains']]
reformat_summarized['No Results'] = [int(i) for i in reformat_summarized['No Results']]
reformat_summarized2 = reformat_summarized[['Drug', 'Main Lineage', 'Number of Resistant Strains', 'Number of Susceptible Strains', 'No Results']].groupby(['Drug', 'Main Lineage']).sum().reset_index()

,Drug,Main Lineage,Number of Resistant Strains,Number of Susceptible Strains,No Results
0,amk,1,0,31,1135
1,amk,2,42,451,1436
2,amk,3,0,55,1484
3,amk,4,9,882,3410
4,amk,5,0,0,15
...,...,...,...,...,...
99,str,4,141,842,3318
100,str,5,0,2,13
101,str,6,0,1,1
102,str,B,2,15,68


In [53]:
reformat_summarized.to_csv('/home/kin672/gentb-summer22/Summarize Sanjana Dataset/summarized_8-2.csv', index = False)

In [ ]:
reformat_summarized2.to_csv('/home/kin672/gentb-summer22/Summarize Sanjana Dataset/summarized_8-2.csv', index = False)